# Inicialización  de  un  modelo  con  pesos  preentrenados

En esta sección, se preparará el modelo que se utilizará para el ajuste fino de la clasificación e identificación de mensajes de spam. Se va a comenzar inicializando el modelo preentrenado con el que se trabajo en la sección05.

![Texto alternativo](./imgs/6.7.png)

In [1]:
import sys
import os

# Obtiene la ruta de la carpeta principal del proyecto (subiendo un nivel desde seccion05)
ruta_proyecto_principal = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añade esta ruta a la lista de lugares donde Python busca módulos
if ruta_proyecto_principal not in sys.path:
    sys.path.append(ruta_proyecto_principal)

In [2]:
from spamDataset import SpamDataset
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

train_dataset = SpamDataset(
    csv_file="CSV/train.csv",
    max_length=None,
    tokenizer=tokenizer
)

In [3]:
#Reeutilizar configuraciones
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"
BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])
assert train_dataset.max_length <= BASE_CONFIG["context_length"], (
    f"Dataset length {train_dataset.max_length} exceeds model's context "
    f"length {BASE_CONFIG['context_length']}. Reinitialize data sets with "
    f"`max_length={BASE_CONFIG['context_length']}`"
)

A  continuación, se importa  la  función  download_and_load_gpt2  del  archivo  gpt_download.py  que  se descargo  en  el  sección 05.  Además,  también se reutilizará  la  clase  GPTModel  y  la  función  load_weights_into_gpt  del  capítulo  05  para  cargar  los  pesos  descargados  en  elModelo  GPT:

In [4]:
from seccion05_PreentrenamientoDatosNoEtiquetados.gpt_download import download_and_load_gpt2
from seccion05_PreentrenamientoDatosNoEtiquetados.loadWeightsGPTModels import load_weights_into_gpt
from seccion04_ImplementacionGPTGeneracionTexto.gptModel import GPTModel

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")
model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval()

File already exists and is up-to-date: gpt2\124M\checkpoint
File already exists and is up-to-date: gpt2\124M\encoder.json
File already exists and is up-to-date: gpt2\124M\hparams.json
File already exists and is up-to-date: gpt2\124M\model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2\124M\model.ckpt.index
File already exists and is up-to-date: gpt2\124M\model.ckpt.meta
File already exists and is up-to-date: gpt2\124M\vocab.bpe


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

In [5]:
from seccion04_ImplementacionGPTGeneracionTexto.generateTextSimple import generate_text_simple
from seccion05_PreentrenamientoDatosNoEtiquetados.trainModelSimple import text_to_token_ids, token_ids_to_text

text_1 = "Every effort moves you"
tokens_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_1, tokenizer=tokenizer),
    max_new_tokens=15,
    context_size=BASE_CONFIG["context_length"]
)

print(token_ids_to_text(tokens_ids,tokenizer))

Every effort moves you forward.

The first step is to understand the importance of your work


Como se puede  ver  en  base  a  la  siguiente  salida,  el  modelo  genera  un  texto  coherente,  lo  cual  es  un  indicador  de  que  los  pesos  del  modelo  se  han  cargado  correctamente.

Ahora,  antes  de  comenzar  a  ajustar  el  modelo  como  clasificador  de  spam,  ver  si  el  modelo  ya  puede  clasificar  mensajes  de  spam  al  darle  instrucciones:

In [6]:
text_2 = (
    "Is the following text 'spam'? Answer with 'yes' or 'no':"
    " 'You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award.'"
)
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_2, tokenizer),
    max_new_tokens=23,
    context_size=BASE_CONFIG["context_length"]
)
print(token_ids_to_text(token_ids, tokenizer))

Is the following text 'spam'? Answer with 'yes' or 'no': 'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.'

The following text 'spam'? Answer with 'yes' or 'no': 'You are a winner


A  juzgar  por  el  resultado,  es  evidente  que  el  modelo  tiene  dificultades  para  seguir instrucciones.
Esto  era  de  esperar,  ya  que  solo  ha  pasado  por  un  entrenamiento  previo  y  le  falta ajuste  de  instrucción.

[Agregar un encabezado de clasificación](./5_encabezado_clasificación.ipynb)

